In [ ]:
import cv2
import os
import numpy as np
import subprocess as sb


In [ ]:
shape = int(224)


In [ ]:
name = input('Enter your name (without space) to label the image folder : ')

In [ ]:
# Parent Directory path 
parent_dir = "/Users/box/Desktop/mlops-ws/photos_dataset/training/"
  
# Path 
path = os.path.join(parent_dir,name)

path

In [ ]:
def fliph(photo):
    ifliph = np.zeros(photo.shape).astype('uint8')
    for i in range(0,photo.shape[0]):
        ifliph[photo.shape[0]-i-1,:] = photo[i,:]
    return ifliph

def flipv(photo):
    iflipv = np.zeros(photo.shape).astype('uint8')
    for i in range(0,photo.shape[0]):
        iflipv[:,i] = photo[:,photo.shape[0]-i-1]
    return iflipv

def r270(photo):
    iflip270 = np.zeros(photo.shape).astype('uint8')
    for i in range(0,photo.shape[0]):
        iflip270[photo.shape[0]-i-1,:] = photo[:,i]
    return iflip270

In [ ]:

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_extractor(photo):
    cord = face_classifier.detectMultiScale(photo)    
    if cord is ():
        return None
    photo = cv2.resize(photo,(shape,shape))
    return photo

cap = cv2.VideoCapture(0)
count = 0

# Collect 200 samples of your face from webcam input to make 1600 images
while True:
    status , photo = cap.read()
    print(status)
    if status == True:
        photo = face_extractor(photo)
        if photo is not None:
            
            scount = count*8 + 1  #for one photo will become 8 photos
            file_name_path = path.format(name) + str(scount) + '.jpg'
            cv2.imwrite(file_name_path,photo)

            scount += 1
            file_name_path = path.format(name) + str(scount) + '.jpg'
            cv2.imwrite(file_name_path,fliph(photo))

            scount += 1
            file_name_path = path.format(name) + str(scount) + '.jpg'
            cv2.imwrite(file_name_path,flipv(photo))

            scount += 1
            file_name_path = path.format(name) + str(scount) + '.jpg'
            cv2.imwrite(file_name_path,r270(photo))

            scount += 1
            file_name_path = path.format(name) + str(scount) + '.jpg'
            cv2.imwrite(file_name_path,fliph(r270(photo)))

            scount += 1
            file_name_path = path.format(name) + str(scount) + '.jpg'
            cv2.imwrite(file_name_path,flipv(r270(photo)))

            scount += 1
            file_name_path = path.format(name) + str(scount) + '.jpg'
            cv2.imwrite(file_name_path,fliph(flipv(r270(photo))))

            scount += 1
            file_name_path = path.format(name) + str(scount) + '.jpg'
            cv2.imwrite(file_name_path,flipv(fliph(photo)))
            
            count += 1

            # Put count on images and display live count
            cv2.putText(photo, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Display', photo)

        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 27 or count == 200: #27 is the Escape Key
            break
            

cap.release()
cv2.destroyAllWindows()

In [ ]:
#Moving 400 images from train to test database

#generate random 400 numbers from 1 to 1600
test = np.random.randint(low=1, high=1600, size=400)
for i in range(0,350):
    command = 'mv /Users/box/Desktop/mlops-ws/photos_dataset/training/pranjali'.format(name) + str(test[i]) + '.jpg ' + '/Users/box/Desktop/mlops-ws/photos_dataset/testing/' + name
    #print(command)
    #command is 'mv source_file destination_folder'
    #change above command as per your os
    sb.getoutput(command)

print("Generating Database complete")


In [ ]:
from keras.applications.vgg16 import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
model = VGG16(weights = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop (2).h5', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [ ]:
model.layers[1].trainable

In [ ]:
for i in model.layers:
    i.trainable = False

In [ ]:
model.layers[1].trainable

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Flatten
from keras.models import Model

In [ ]:
model.output

In [ ]:
top_model = model.output

top_model = Flatten(name = "flatten")(top_model)
top_model = Dense(256,activation='relu')(top_model)
top_model = Dense(2, activation = "sigmoid")(top_model)

In [ ]:
#compilig the new model
newmodel = Model(inputs=model.input,outputs=top_model)

#newmodel.layers
newmodel.output

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/Users/box/Desktop/mlops-ws/photos_dataset/training/'
validation_data_dir = '/Users/box/Desktop/mlops-ws/photos_dataset/testing/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
train_generator.class_indices

In [ ]:
validation_generator.class_indices

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("nface_classifier.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
newmodel.compile(loss = 'binary_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])

nb_train_samples = 2565
nb_validation_samples = 635
epochs = 3
batch_size = 16

history = newmodel.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

newmodel.save("nface_classifier.h5")

In [ ]:
import cv2
import numpy as np

In [ ]:
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import preprocess_input

In [ ]:
from keras.models import load_model
classifier = load_model('nface_classifier.h5')

In [ ]:
cap = cv2.VideoCapture(0)
status , image = cap.read()
print(status)
nimage = cv2.resize(image,(224,224))
aimage = np.expand_dims(nimage, axis=0)
finalimage = preprocess_input(aimage)
pred = classifier.predict(finalimage)
name = 'person'
if pred[0][0] == 1.0:
    name = 'Pranjali'
else :
    name = 'not found'
cv2.putText(image, name, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
cv2.imshow('livestream',image)
if cv2.waitKey() == 27:
    cv2.destroyAllWindows()
cap.release()
